### This version saves paradigm parameters and does not include some summary variables that can be computed from the main info:
    reversal, post_err, score_total, hit_total, block_acc

In [1]:
%matplotlib inline
import sys 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as io
import pickle

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [122]:
SBJ = 'IR84'

In [123]:
prj_dir = '/Volumes/hoycw_clust/PRJ_Error/'
results_dir = prj_dir+'results/'
fig_type = '.png'
data_dir = prj_dir+'data/'
sbj_dir  = data_dir+SBJ+'/'
# paths = {'Rana': '/Users/colinhoy/Code/PRJ_Error/data/logs/',
#          'Adi': '/Users/colinhoy/Code/PRJ_Error/data/logs/',
#          'IR57': '/Users/colinhoy/Code/PRJ_Error/data/logs/'}
logs = {'Rana_1.6': 'Rana2_response_log_20170321103129_DATA.txt',
        'Adi_1.7': 'adi_response_log_20170321153641.txt',
        'CP22': '222_response_log_20170609140407.txt',
        'CP23': '223_response_log_20170930123015.txt',
        'CP241': '224-1_response_log_20171206121023.txt',
        'CP242': 'cp24_2_response_log_20171209120902.txt',
        'CP25': 'CP25_response_log_20180811082953.txt',
        'IR57': '857_response_log_20170322112243_CWHedit.txt',#CWHedit added n_training, n_examples lines
        'IR60': '60_response_log_20170613100307.txt',
        'IR62': 'ir62_response_log_20170713124719.txt',
        'IR63': 'IR63_response_log_20170921095757.txt',
        'IR65': '865_response_log_20171207130759.txt',
        'IR66': 'ir66_response_log_20171219124409.txt',
        'IR67': 'ir673_response_log_20180124103600.txt',
        'IR68': 'IR68_response_log_20180124140950.txt',
        'IR69': '869p2_response_log_20180211111609.txt',#this is 2nd run, another one before!
        'IR71': '71_response_log_20180221115510.txt',
        'IR72': 'Ir72_response_log_20180325133520.txt',#2 log files, this has easy, other has hard blocks
        'IR74': 'ir742_response_log_20180327170333.txt',
        'IR75': 'IR75_response_log_20180531221813.txt',
        'IR76': 'IR76_response_log_20180603181356.txt',
        'IR77': '877_response_log_20180620121202.txt',#1/3 logs, has 2 easy, 8773 has 2 hards
        'IR78': 'IR78_response_log_20180628052715.txt',
        'IR79': 'IR79_response_log_20180710112314.txt',
        'IR82': 'IR82_response_log_20180928162311.txt',
        'IR84': 'IR84_response_log_20181025094454.txt',
        'P1': 'Pilot1_2_response_log_20170412131644.txt',
        'P2': 'pilot2_response_log_20170412140615.txt',
        'P3': 'pilot3_response_log_20170413110228.txt',
        'P4': 'Pilot4_2_response_log_20170418140941.txt',
        'P5': 'colin_real_response_log_20170412103113.txt',
        'P6': 'pilot_adi_response_log_20170414122257.txt',
        'P7': 'pilot_Rana_response_log_20170415155844.txt',
        'P8': 'Giao_response_log_20170419161340.txt',
        'P9': 'Sundberg_response_log_20170419150222.txt',
        'colin_vec': 'colin_circle_wVec_response_log_20171222141248.txt',
        'colin_novec': 'colin_noVec_response_log_20171222142110.txt'
       }

### Load SBJ Data

In [124]:
log_filename = os.path.join(sbj_dir,'00_raw',logs[SBJ])

log_file = open(log_filename,'r')
log = log_file.readlines()
log_file.close()

### Process Version-Specific Parameters

In [125]:
prdm = {}
for line in log:
    # Script version
    if line.find('paradigm_name =')!=-1:
        prdm['prdm_name'] = line[line.find('= ')+2:line.find('\n')]
    if line.find('paradigm_version =')!=-1:
        prdm['prdm_version'] = line[line.find('= ')+2:line.find('\n')]

    # Timing variables
    if line.find('interval_dur =')!=-1:
        prdm['target'] = float(line[line.find('= ')+2:line.find('\n')])
    if line.find('feedback_delay =')!=-1:
        prdm['fb_delay'] = float(line[line.find('= ')+2:line.find('\n')])
    if line.find('feedback_dur =')!=-1:
        prdm['fb'] = float(line[line.find('= ')+2:line.find('\n')])
    
    # ITIs and boundaries between them
    if line.find('ITIs')!=-1:
        prdm['ITIs'] = [float(string) \
                    for string in line[line.find('[')+1:line.find(']')].split(',')]
        ITI_bounds = np.mean([prdm['ITIs'][:-1], prdm['ITIs'][1:]],0)
        
    # Tolerance limits/clamps
    if line.find('tolerance_lim')!=-1:
        prdm['tol_lim'] = [float(string) \
                         for string in line[line.find('[')+1:line.find(']')].split(',')]
        
    # Trial count variables
    if line.find('n_blocks')!=-1:
        prdm['n_blocks'] = int(line[line.find('=')+2:])
    if line.find('n_trials')!=-1:
        prdm['n_trials'] = int(line[line.find('=')+2:])
    if line.find('n_examples')!=-1:
        prdm['n_examples'] = int(line[line.find('=')+2:])
    if line.find('n_training')!=-1:
        prdm['n_training'] = int(line[line.find('=')+2:])

prdm['trl_len'] = prdm['target']+\
            prdm['fb_delay']+prdm['fb']

### Save paradigm parameters

In [126]:
# Python readable
prdm_fname = os.path.join(sbj_dir,'03_events',SBJ+'_prdm_vars.pkl')
with open(prdm_fname, 'wb') as f:
    pickle.dump(prdm, f, pickle.HIGHEST_PROTOCOL)
# MATLAB Readable
prdm_fname = os.path.join(sbj_dir,'03_events',SBJ+'_prdm_vars.mat')
io.savemat(prdm_fname,prdm)

In [127]:
print 'paradigm: ', prdm['prdm_name'], ' v', prdm['prdm_version']
print
print 'interval: ', prdm['target'], 's'
print 'feedback_delay: ', prdm['fb_delay'], 's'
print 'feedback duration: ', prdm['fb'], 's'
print 'total trial length: ', prdm['trl_len'], 's'
print
print 'n_blocks: ', prdm['n_blocks']
print 'n_trials/block: ', prdm['n_trials']
print 'n_full_vis_examples: ', prdm['n_examples']
print 'n_training/condition: ', prdm['n_training']
print
# ITI_bounds = [np.mean(a,b) for a, b in zip(ITIs[:-1],ITIs[1:])]
print 'ITIs:',prdm['ITIs'], ITI_bounds
print 'tolerance_lim:', prdm['tol_lim']

paradigm:  target_time_cyclone  v 2.4.5

interval:  1.0 s
feedback_delay:  0.8 s
feedback duration:  1.0 s
total trial length:  2.8 s

n_blocks:  2
n_trials/block:  75
n_full_vis_examples: 

KeyError: 'n_examples'

### Extract Trial Info

In [128]:
resp_lines = [line for line in log if line.find('Outcome=')!=-1]
data = pd.DataFrame({'Block': [line[line.find('B')+1] for line in resp_lines],
                     'Trial': [int(line[line.find('_T')+2:line.find(':')]) for line in resp_lines],
                     'Condition': [line[line.find('_type')+8:line.find('_type')+12] for line in resp_lines],
                     'Hit': [line.count('WIN') for line in resp_lines],
                     'RT': [line[line.find('RT')+5:line.find('RT')+5+13].strip() for line in resp_lines],
                     'Tolerance': [float(line[line.find('tol')+12:line.find('\n')]) for line in resp_lines],
                     'Timestamp': [float(line[:line.find('.')+4]) for line in resp_lines]})
data['Score'] = [100 if data['Hit'][ix]==1 else -100 for ix in range(len(data))]

# Calculate ITIs
data['ITI'] = [data['Timestamp'][ix]-data['Timestamp'][ix-1]-prdm['trl_len'] if ix!=0 else 0 \
               for ix in range(len(data))]
data.loc[data['Trial']==0,'ITI'] = 0
data.loc[data['Block']==-1,'ITI'] = 0
# Match real ITIs to ITI categories
ITI_bin_edges = np.insert(ITI_bounds, ITI_bounds.shape[0], prdm['ITIs'][-1]+0.1)
ITI_bin_edges = np.insert(ITI_bin_edges, 0, 0)
data['ITI type'] = [prdm['ITIs'][np.argmax(np.histogram(data['ITI'][ix],bins=ITI_bin_edges)[0])]\
        if data['ITI'][ix]!=0 else 0 for ix in range(len(data))]
# if len(prdm['ITIs'])==4:    # target_time v1.8.5+
# elif len(prdm['ITIs'])==3:  # target_time v1.8.4 and below
# else:               # Errors for anything besides len(ITIs)==3,4
#     assert len(prdm['ITIs'])==4
            
# Fix Reversals, Block, and missed RTs
for ix in range(len(data)):
    if data['RT'][ix][0:2]=='-1':#No response
        data.loc[ix,'RT'] = -1
        data.loc[ix,'Score'] = 0            # !!! may change depending on version !!!!
    else:# Real Responses
        if data['RT'][ix].find(';')!=-1:# shorter number of digits, clip ';'
            data.loc[ix,'RT'] = float(data['RT'][ix][:data['RT'][ix].find(';')])
        else:
            data.loc[ix,'RT'] = float(data['RT'][ix])
    if data['Block'][ix]=='T':# Training
        data.loc[ix,'Block'] = -1
    else:
        data.loc[ix,'Block'] = int(data['Block'][ix])

In [129]:
# print(SBJ,' n_trials = ',len(data))
# for cond in data['Condition'].unique():
#     print('==============', cond, '==============')
#     print('Accuracy:', data[data['Condition']==cond]['Hit'].mean())
#     print('--> correct = ', data[data['Condition']==cond]['Hit'].sum(),\
#           '/', len(data[data['Condition']==cond]['Hit']))
#     print('n_reverse:', data[data['Condition']==cond]['Reversal'].sum())

### Save Behavioral Data

In [130]:
behav_fname = os.path.join(sbj_dir,'03_events',SBJ+'_behav.csv')

data.to_csv(behav_fname,index_label='Total_Trial')

In [131]:
# pd.set_option('max_rows', 75)
# data[data['Block']==1]
